In [1]:
from texascovid19 import constants
import requests
import pandas as pd
from datetime import date

Fetch each day's data.  The data is "current as of 8 p.m. the day before reporting."

In [2]:
r = requests.get(constants.DSHS_XHR_URL, verify=False)
r.raise_for_status()

/Users/dan/code/texascovid19/venv/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'services5.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [3]:
county_data = r.json()['features']

In [4]:
norm_county_data = []
for county in county_data:
        norm_county_data.append({
            'County of Residence': county['attributes']['County'],
            'Deaths': county['attributes']['Deaths'] or 0,
            'Number of Cases': county['attributes']['Count_']
        })

In [5]:
df = pd.DataFrame(data=norm_county_data)

In [6]:
df

,County of Residence,Deaths,Number of Cases
0,Harris,4,563
1,Dallas,11,549
2,Travis,1,206
3,Denton,2,191
4,Tarrant,1,175
...,...,...,...
117,San Jacinto,0,1
118,San Patricio,0,1
119,Swisher,0,1
120,Wise,0,1


In [7]:
df.set_index("County of Residence", inplace=True)

In [8]:
today = date.today().isoformat()

In [9]:
df.to_csv(f"{constants.DATA_PATH}/{today}.csv", header=True, index=True)

Append today's data to timeseries

In [10]:
df_timeseries = pd.read_csv(f"{constants.TIMESERIES_CSV}").set_index("County of Residence")

In [11]:
df.rename(columns={"Number of Cases": today}, inplace=True)

In [12]:
df.drop("Deaths", axis=1, inplace=True)

In [13]:
if today in df_timeseries.columns:
    df_timeseries.drop(columns=today, inplace=True)

In [14]:
df_timeseries = pd.concat([df_timeseries, df], axis=1).rename_axis('County of Residence')

In [15]:
df_timeseries = df_timeseries.fillna(0).convert_dtypes(convert_integer=True)

In [16]:
df_timeseries.to_csv(constants.TIMESERIES_CSV, header=True, index=True)